In [1]:
import json
import matplotlib.pyplot as plt

from src.data_processing.data_loader import data_loader
import src.training.training_utils as training_utils
# from src.results.main import evaluate
# import src.visualisation.main as vis_main


In [163]:
with open("src/training/run_config.json", "r") as f:
    run_config = json.load(f)
    f.close()

# Extract info
data_config = run_config["data_config"]
model_config = run_config["model_config"]
training_config = run_config["training_config"]

In [3]:
print(run_config)

{'data_config': {'data_name': 'MIMIC', 'target_window': 12, 'feat_set': 'vit-sta', 'time_range': [0, 10], 'train_test_ratio': 0.6, 'train_val_ratio': 0.4, 'seed': 2323}, 'model_config': {'model_name': 'DirVRNN', 'latent_size': 10, 'gate_layers': 2, 'gate_nodes': 20, 'feat_extr_layers': 2, 'feat_extr_nodes': 20, 'num_clus': 2}, 'training_config': {'lr': 0.001, 'epochs': 5, 'bs': 128}}


In [5]:
from importlib import reload
import src.data_processing.data_loader as loader
reload(loader)

<module 'src.data_processing.data_loader' from 'C:\\Users\\hruia\\PycharmProjects\\VarPhenClustering\\src\\data_processing\\data_loader.py'>

In [6]:
data_info = loader.data_loader(**data_config);

C:\Users\hruia\PycharmProjects\VarPhenClustering\src\data_processing\data_loading_utils.py:453: FutureWarning: is_monotonic is deprecated and will be removed in a future version. Use is_monotonic_increasing instead.
  cond1 = X[self.id_col].is_monotonic



MIMIC data has been subsettted to the following features: 
 ['DBP', 'ESI', 'HR', 'RR', 'SBP', 'SPO2', 'TEMP', 'age', 'gender'].


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8911/8911 [00:34<00:00, 261.31it/s]



MIMIC data successfully loaded.

Basic information 
 Input shape: (8911, 10, 9), (8911, 4) 
 Outcome Distribution: De      11.0
I     1132.0
W     6784.0
Di     984.0
dtype: float64


In [187]:
from src.models.Dir_VRNN import Dir_VRNN_train as model_train
from src.models.Dir_VRNN import model as model
from src.models.Dir_VRNN import Dir_VRNN_utils as model_utils

reload(model_train);
reload(model);
reload(model_utils);

run_config["training_config"]["bs"] = 128
run_config["training_config"]["epochs"] = 20
run_config["model_config"]["num_clus"] = 3
model = model_train.DirVRNN(data_info = data_info, model_config=model_config, training_config=training_config);

In [188]:
run_config

{'data_config': {'data_name': 'MIMIC',
  'target_window': 12,
  'feat_set': 'vit-sta',
  'time_range': [0, 10],
  'train_test_ratio': 0.6,
  'train_val_ratio': 0.4,
  'seed': 2323},
 'model_config': {'model_name': 'DirVRNN',
  'latent_size': 10,
  'gate_layers': 2,
  'gate_nodes': 20,
  'feat_extr_layers': 2,
  'feat_extr_nodes': 20,
  'num_clus': 3},
 'training_config': {'lr': 0.001, 'epochs': 20, 'bs': 128}}

In [189]:
model.fit()

Train epoch: 1   [1837264.41912 - 100%]
Val epoch: 1   [1312438.37500 - 0%]
Train epoch: 2   [774340.64338 - 100%]
Val epoch: 2   [95147.15625 - 0%]
Train epoch: 3   [16349.21591 - 100%]
Val epoch: 3   [-423.29688 - 0%]
Train epoch: 4   [-430.82109 - 100%]
Val epoch: 4   [-439.46552 - 0%]
Train epoch: 5   [-446.51093 - 100%]
Val epoch: 5   [-455.92923 - 0%]
Train epoch: 6   [-478.51230 - 100%]
Val epoch: 6   [-578.17712 - 0%]
Train epoch: 7   [-621.24868 - 100%]
Val epoch: 7   [-642.68951 - 0%]
Train epoch: 8   [-650.71106 - 100%]
Val epoch: 8   [-658.13068 - 0%]
Train epoch: 9   [-663.30345 - 100%]
Val epoch: 9   [-667.90851 - 0%]
Train epoch: 10   [-671.20017 - 100%]
Val epoch: 10   [-674.07709 - 0%]
Train epoch: 11   [-675.67582 - 100%]
Val epoch: 11   [-676.95667 - 0%]
Train epoch: 12   [-677.45951 - 100%]
Val epoch: 12   [-677.80994 - 0%]
Train epoch: 13   [-678.00172 - 100%]
Val epoch: 13   [-678.22363 - 0%]
Train epoch: 14   [-678.41031 - 100%]
Val epoch: 14   [-678.57233 - 0%]


In [169]:
 
    
# Predict model
X_test, y_test = data_info["X"][-1], data_info["y"][-1]
history = model.predict(X_test, y_test)
     
# Print available keys
print(history.keys())
             
                 
                     
                         
                             
                                 
                                     
                                         
                                             
                                                 
                                                    

Test: [-1133.16125 - 0%]
dict_keys(['alpha_prior', 'alpha_enc', 'est_pi', 'est_cluster_means', 'est_outcomes', 'est_gen_mean', 'est_gen_data', 'cell_state', 'pred_loss', 'y_pred', 'X', 'y'])


In [171]:
history["est_pi"].shape

torch.Size([3565, 10, 6])

In [179]:
test = history["est_pi"][:, -1, :]
clus_one_hot = [torch.argmax(test, axis=1).detach().numpy()]

In [186]:
np.matmul(np.transpose(clus_one_hot), y_test)

array([[  1.,  92., 608.,  91.],
       [  2.,  94., 540.,  62.],
       [  0.,  89., 530.,  72.],
       [  0.,   0.,   0.,   0.],
       [  1.,  98., 522.,  79.],
       [  0.,  80., 514.,  90.]])

In [42]:
import numpy as np
history["alpha_enc"][0, :, :] / torch.sum(history["alpha_enc"][0, :, :], dim=-1, keepdims=True)

tensor([[0.3198, 0.6802],
        [0.3029, 0.6971],
        [0.3031, 0.6969],
        [0.3030, 0.6970],
        [0.3031, 0.6969],
        [0.3031, 0.6969],
        [0.3031, 0.6969],
        [0.3030, 0.6970],
        [0.3030, 0.6970],
        [0.3030, 0.6970]], grad_fn=<DivBackward0>)

In [43]:
history["alpha_enc"][101, :, :] / torch.sum(history["alpha_enc"][101, :, :], dim=-1, keepdims=True)

tensor([[0.3194, 0.6806],
        [0.3027, 0.6973],
        [0.3031, 0.6969],
        [0.3030, 0.6970],
        [0.3031, 0.6969],
        [0.3032, 0.6968],
        [0.3020, 0.6980],
        [0.3020, 0.6980],
        [0.3020, 0.6980],
        [0.3020, 0.6980]], grad_fn=<DivBackward0>)

In [38]:
import torch
torch.min(history["est_pi"], dim=1)

torch.return_types.min(
values=tensor([[0.3029, 0.6802],
        [0.3025, 0.6785],
        [0.3039, 0.6778],
        ...,
        [0.3020, 0.6792],
        [0.3020, 0.6800],
        [0.3014, 0.6812]], grad_fn=<MinBackward0>),
indices=tensor([[1, 0],
        [3, 0],
        [1, 0],
        ...,
        [8, 0],
        [8, 0],
        [8, 0]]))